In [1]:
import plotly.express as px
import pandas as pd

# Average Duration of Activities by Day and Mood
# Load your data
df = pd.read_csv("C:\\Users\\Gabriel\\Desktop\\4TH YEAR\\Data Analytics\\IndividualDashboard\\Tracker1.csv")

# Group the data by 'Day of the Week' and 'How I feel' and calculate the average duration
heatmap_data = df.groupby(['Day of the Week', 'How I feel'])['Duration (mins)'].mean().unstack()

# Custom order for days of the week
day_order = ['Sunday', 'Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday']

# Sort the heatmap data according to the custom order
heatmap_data = heatmap_data.reindex(day_order)

# Create the interactive heatmap with tilted x-axis and y-axis labels
fig = px.imshow(heatmap_data.fillna(0),
                labels=dict(x="Mood", y="Day of the Week", color="Average Duration"),
                x=heatmap_data.columns,
                y=heatmap_data.index,
                title="")

# Update the layout to add axis labels and tilt the labels
fig.update_layout(xaxis_title='Mood', yaxis_title='Day of the Week',
                  xaxis_tickangle=35)  # Adjust the angle as needed

# Show the plot
fig.show()

In [2]:
df['Start Time'] = pd.to_datetime(df['Start Time'], format='mixed')

# Create the scatter plot
fig1 = px.scatter(df, x='Start Time', y='Duration (mins)', color='Activity Description',
                  title='', labels={'Start Time': 'Time', 'Duration (mins)': 'Duration (minutes)'},
                  color_discrete_sequence=px.colors.qualitative.Pastel)
fig1.show()

c:\Users\Gabriel\anaconda3\Lib\site-packages\_plotly_utils\basevalidators.py:106: FutureWarning:

The behavior of DatetimeProperties.to_pydatetime is deprecated, in a future version this will return a Series containing python datetime objects instead of an ndarray. To retain the old behavior, call `np.array` on the result



In [3]:
# Group by 'Activity Description' and sum the 'Duration (mins)' for each activity
# Total Duration of Top 10 Activities
total_duration_per_activity = df.groupby('Activity Description')['Duration (mins)'].sum().reset_index()

# Sort data by duration and take the top 10 activities
top_10_duration_per_activity = total_duration_per_activity.sort_values(by='Duration (mins)', ascending=False).head(10)

# Create the interactive bar chart
fig2 = px.bar(top_10_duration_per_activity, x='Activity Description', y='Duration (mins)',
                             title='',
                             labels={'Activity Description': 'Activity', 'Duration (mins)': 'Total Duration (minutes)'},
                             color='Duration (mins)', color_continuous_scale='Viridis')

# Show the plot
fig2.show()

In [4]:
# Group by 'Activity Description' and count the occurrences
activity_counts = df['Activity Description'].value_counts().reset_index()

# Rename the columns for clarity
activity_counts.columns = ['Activity', 'Count']

# Select the top 10 most common activities
top_10_activities = activity_counts.head(10)

# Create an interactive bar chart for the top 10 activities
fig3 = px.bar(top_10_activities, x='Activity', y='Count',
              title='Top 10 Most Common Activities',
              labels={'Activity': 'Activity', 'Count': 'Count'})

# Add axis labels and rotate x-axis labels for better readability
fig3.update_layout(xaxis_title='Activity', yaxis_title='Count')
fig3.update_xaxes(tickangle=35)

# Show the interactive bar chart
fig3.show()

In [5]:
# Group the data by 'How I feel' and count the occurrences
mood_counts = df['How I feel'].value_counts().reset_index()

# Rename the columns for clarity
mood_counts.columns = ['Mood', 'Count']

# Sort the data by count and select the top 5 moods
top_5_moods = mood_counts.head(5)

# Create an interactive pie chart for the top 5 moods
fig4 = px.pie(top_5_moods, names='Mood', values='Count',
              title='Top 5 Moods Across All Activities',
              labels={'Mood': 'Mood', 'Count': 'Count'})

# Add labels and legends
fig4.update_traces(textposition='inside', textinfo='percent+label')

# Show the interactive pie chart for the top 5 moods
fig4.show()

In [6]:
# Create a hierarchical dataframe for the Sunburst chart
# Sunburst Chart of Activities by Day of the Week
sunburst_data = df.groupby(['Day of the Week', 'Activity Description'])['Duration (mins)'].sum().reset_index()

# Create an interactive Sunburst chart
fig_sunburst = px.sunburst(sunburst_data, path=['Day of the Week', 'Activity Description'], values='Duration (mins)',
                           title='',
                           labels={'Duration (mins)': 'Total Duration (minutes)'})

# Show the interactive Sunburst chart
fig_sunburst.show()

In [8]:
import dash
import dash_bootstrap_components as dbc
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output

# Define color palette
colors = {
    'background': '#f4f4f4',
    'text': '#34495e',
    'plot_background': '#ffffff',
    'accent_color': '#3498db',
}

# Define the Dash app with Bootstrap
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])

# Styling
font_style = {
    'fontFamily': 'Arial, sans-serif',
    'color': colors['text']
}

# Define the app layout
app.layout = dbc.Container(fluid=True, style={'backgroundColor': colors['background'], 'padding': '10px', 'marginBottom': '0px', **font_style}, children=[
    dbc.Row([
        dbc.Col(html.H1("Data Visualization", style={'color': 'blue', 'textAlign': 'center'}), width={'size': 12})
    ]),
    dbc.Row([
        dbc.Col(html.H2("Submitted by: John Gabriel Rago", style={'textAlign': 'center', 'fontSize': '16px'}), width={'size': 12})
    ]),
    dbc.Row([
        dbc.Col([
            html.A("Open in localhost", href="http://127.0.0.1:8050", target="_blank", className='btn btn-info mb-3'),
            
            # Sidebar for plot type selection and plot dimensions
            dcc.RadioItems(
                id='plot-type',
                options=[
                    {'label': ' Heatmap', 'value': 'heatmap'},
                    {'label': ' Scatter Plot', 'value': 'scatterplot'},
                    {'label': ' Bar Chart 1', 'value': 'barchart1'},
                    {'label': ' Bar Chart 2', 'value': 'barchart2'},
                    {'label': ' Pie Chart', 'value': 'piechart'},
                    {'label': ' Sunburst Chart', 'value': 'sunburst'}
                ],
                value='heatmap',
                labelStyle={'display': 'block', 'marginBottom': '15px', 'padding': '10px', 'backgroundColor': colors['plot_background'], **font_style},
            ),
            
            # Add controls to adjust plot dimensions
            html.Label("Adjust Plot Dimensions", style={'color': colors['text'], 'fontSize': '16px'}),
            dcc.Slider(id='plot-height-slider', min=300, max=800, step=50, value=600,
                       marks={300: '300', 500: '500', 600: '600', 700: '700', 800: '800'},
                       tooltip={'placement': 'bottom', 'always_visible': True}),
            dcc.Slider(id='plot-width-slider', min=500, max=1200, step=50, value=800,
                       marks={500: '500', 800: '800', 1000: '1000', 1200: '1200'},
                       tooltip={'placement': 'bottom', 'always_visible': True}),
            
        ], width=3, style={'padding': '20px', 'backgroundColor': colors['plot_background']}),

        dbc.Col([
            dcc.Graph(
                id='plot',
                className='plot-background',
            ),
            
            # Add a hoverinfo box for the scatter plot
            html.Div(id='hover-data', style={'color': colors['text'], 'fontSize': '14px', 'marginTop': '10px'})
            
        ], width=9),

    ]),
    dbc.Row([
        dbc.Col([
            html.Div([
                html.H4("Dashboard Description", style={'color': colors['accent_color']}),
                html.P("A glimpse into the daily activities of myself throughout the course of 12 days. During this time period, my routines, activities, and emotions are detailed in this dashboard. The inclusion of timestamps, activity descriptions, and my personal affective evaluations provides a distinctive insight into my day-to-day activities. Embark on an exploration of the patterns and emotions that delineate my world as I recount my daily activities in great detail. This dashboard was built from scratch using bootstrap functionalities and plotly express dash library and for plot visual representations.You can visit my repository and follow the readme.txt instructions to use and modify this dashboard."),
                html.A("Visit my GitHub Repository", href="https://github.com/gabrielrago/IndividualDashboard", target="_blank", className='btn btn-primary')
            ], style={'padding': '20px', 'backgroundColor': colors['plot_background'], 'marginBottom': '10px', **font_style})
        ])
    ])
])

# Callback for updating the plots and dimensions
@app.callback(
    [Output('plot', 'figure'),
     Output('plot', 'style'),
     Output('hover-data', 'children')],
    [Input('plot-type', 'value'),
     Input('plot-height-slider', 'value'),
     Input('plot-width-slider', 'value')]
)

def update_graph(plot_type, plot_height, plot_width):
    # Update global plot dimensions
    global prev_plot_height, prev_plot_width
    prev_plot_height = plot_height
    prev_plot_width = plot_width
    
    # Initialize fig with a default Plotly figure
    fig = px.scatter()

    if plot_type == 'heatmap':
        # Heatmap
        fig = px.imshow(heatmap_data.fillna(0),
                labels=dict(x="Mood", y="Day of the Week", color="Average Duration"),
                x=heatmap_data.columns,
                y=heatmap_data.index,
                title="Average Duration of Activities by Day and Mood")

        # Update the layout to add axis labels, tilt the labels, and set the width
        fig.update_layout(xaxis_title='Mood', yaxis_title='Day of the Week',
                          xaxis_tickangle=35, width=plot_width)
    
    elif plot_type == 'scatterplot':
        # Scatter Plot
        fig = px.scatter(df, x='Start Time', y='Duration (mins)', color='Activity Description',
                  title='Activity Duration Over Time', labels={'Start Time': 'Time', 'Duration (mins)': 'Duration (minutes)'},
                  color_discrete_sequence=px.colors.qualitative.Pastel)

        # Add hoverinfo box
        fig.update_traces(hoverinfo=['x', 'y', 'text'])
    
    elif plot_type == 'barchart1':
        # Bar Chart 1
        fig = px.bar(top_10_duration_per_activity, x='Activity Description', y='Duration (mins)',
                             title='Total Duration of Top 10 Activities',
                             labels={'Activity Description': 'Activity', 'Duration (mins)': 'Total Duration (minutes)'},
                             color='Duration (mins)', color_continuous_scale='Viridis')

        # Set the width
        fig.update_layout(width=plot_width)
    
    elif plot_type == 'barchart2':
        # Bar Chart 2
        fig = px.bar(top_10_activities, x='Activity', y='Count',
              title='Top 10 Most Common Activities',
              labels={'Activity': 'Activity', 'Count': 'Count'})

        # Add axis labels and rotate x-axis labels for better readability
        fig.update_layout(xaxis_title='Activity', yaxis_title='Count', width=plot_width)
    
    elif plot_type == 'piechart':
        # Pie Chart
        fig = px.pie(top_5_moods, names='Mood', values='Count',
              title='Top 5 Moods Across All Activities',
              labels={'Mood': 'Mood', 'Count': 'Count'})

        # Add labels and legends
        fig.update_traces(textposition='inside', textinfo='percent+label')
    
    elif plot_type == 'sunburst':
        # Sunburst Chart
        fig = px.sunburst(sunburst_data, path=['Day of the Week', 'Activity Description'], values='Duration (mins)',
                           title='Activities by Day of the Week',
                           labels={'Duration (mins)': 'Total Duration (minutes)'})

    # Set the style of the plot
    plot_style = {'height': plot_height}

    # Define hover data for scatter plot
    hover_data = f"Hovered Data: x = {plot_width}, y = {plot_height}"

    return fig, plot_style, hover_data

# Run the app
# CTRL + CLICK THE BUTTON NAMED OPEN IN LOCAL HOST
if __name__ == '__main__':
    app.run_server(debug=True)